In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from gekko import GEKKO

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("MPC-V3.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DP
from Logic import logic_bat, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)
from MPC import MPC, MPC_carb, MPC_price_carb, MPCModel

In [27]:
df = merge('h16')
h16 = MPCModel('h16')
h16.MPCopt_price(df,start_time='2022-06-19 00:00:00', end_time = '2022-06-19 23:00:00',ini_bat_state=0)

,prod,cons,yield,charge,state,surplus,price,costs,cumm_costs,em_rate,emission,cumm_em
time,,,,,,,,,,,,
2022-06-19 00:00:00,0.0,5.5,-5.5,0.0,0.0,-5.5,1.5,8.5,8.5,0.1,0.8,0.8
2022-06-19 01:00:00,0.0,5.3,-5.3,0.0,0.0,-5.3,1.4,7.5,16.0,0.1,0.5,1.3
2022-06-19 02:00:00,0.0,4.2,-4.2,0.0,0.0,-4.2,1.2,5.2,21.1,0.1,0.6,1.9
2022-06-19 03:00:00,0.0,3.8,-3.8,0.0,0.0,-3.8,1.0,4.0,25.1,0.2,0.6,2.5
2022-06-19 04:00:00,0.0,3.8,-3.8,0.0,0.0,-3.8,0.9,3.4,28.5,0.2,0.6,3.1
2022-06-19 05:00:00,0.0,3.4,-3.4,3.5,0.0,-6.9,0.8,5.7,34.2,0.2,1.3,4.4
2022-06-19 06:00:00,0.1,3.6,-3.5,-3.5,3.5,0.0,0.8,-0.0,34.2,0.2,-0.0,4.4
2022-06-19 07:00:00,0.2,3.3,-3.1,0.0,0.0,-3.1,0.7,2.3,36.5,0.3,0.8,5.2
2022-06-19 08:00:00,0.6,3.9,-3.3,7.0,0.0,-10.3,0.7,6.9,43.4,0.3,2.9,8.0


In [28]:
nf = merge('h28')
h28 = MPCModel('h28')
h28.MPCopt_price(nf,start_time='2022-06-19 00:00:00', end_time = '2022-06-19 23:00:00',ini_bat_state=10)

,prod,cons,yield,charge,state,surplus,price,costs,cumm_costs,em_rate,emission,cumm_em
time,,,,,,,,,,,,
2022-06-19 00:00:00,0.0,4.8,-4.8,-4.8,10.0,0.0,1.5,-0.0,-0.0,0.1,-0.0,-0.0
2022-06-19 01:00:00,0.0,3.3,-3.3,-3.3,5.2,0.0,1.4,-0.0,-0.0,0.1,-0.0,-0.0
2022-06-19 02:00:00,0.0,3.3,-3.3,-1.9,1.9,-1.4,1.2,1.7,1.7,0.1,0.2,0.2
2022-06-19 03:00:00,0.0,2.8,-2.8,0.0,0.0,-2.8,1.0,2.9,4.6,0.2,0.4,0.6
2022-06-19 04:00:00,0.0,3.0,-3.0,0.0,0.0,-3.0,0.9,2.7,7.3,0.2,0.5,1.1
2022-06-19 05:00:00,0.0,2.7,-2.7,2.6,0.0,-5.3,0.8,4.4,11.7,0.2,1.0,2.1
2022-06-19 06:00:00,0.1,2.7,-2.6,-2.6,2.6,0.0,0.8,-0.0,11.7,0.2,-0.0,2.1
2022-06-19 07:00:00,0.1,2.5,-2.4,0.0,0.0,-2.4,0.7,1.8,13.5,0.3,0.6,2.7
2022-06-19 08:00:00,0.3,2.9,-2.6,6.9,0.0,-9.5,0.7,6.4,19.9,0.3,2.6,5.3


In [29]:
tf = merge('k28')
k28 = MPCModel('k28')
k28.MPCopt_price(tf,start_time='2022-06-19 00:00:00', end_time = '2022-06-19 23:00:00',ini_bat_state=0)

,prod,cons,yield,charge,state,surplus,price,costs,cumm_costs,em_rate,emission,cumm_em
time,,,,,,,,,,,,
2022-06-19 00:00:00,0.0,7.3,-7.3,6.3,0.0,-13.6,1.5,20.9,20.9,0.1,1.9,1.9
2022-06-19 01:00:00,0.0,6.3,-6.3,-6.3,6.3,0.0,1.4,-0.0,20.9,0.1,-0.0,1.9
2022-06-19 02:00:00,0.0,5.6,-5.6,0.0,0.0,-5.6,1.2,6.9,27.8,0.1,0.8,2.7
2022-06-19 03:00:00,0.0,4.1,-4.1,0.0,0.0,-4.1,1.0,4.3,32.1,0.2,0.6,3.3
2022-06-19 04:00:00,0.0,4.1,-4.1,3.5,0.0,-7.6,0.9,6.8,38.9,0.2,1.2,4.5
2022-06-19 05:00:00,0.1,3.6,-3.5,-3.5,3.5,0.0,0.8,-0.0,38.9,0.2,-0.0,4.5
2022-06-19 06:00:00,0.0,3.2,-3.2,0.0,0.0,-3.2,0.8,2.7,41.6,0.2,0.8,5.3
2022-06-19 07:00:00,0.2,4.5,-4.3,0.0,0.0,-4.3,0.7,3.2,44.8,0.3,1.1,6.4
2022-06-19 08:00:00,0.6,3.4,-2.8,7.0,0.0,-9.8,0.7,6.6,51.4,0.3,2.7,9.1
